In [ ]:
pwd

'/Users/kiranbijoy/Sem_3/Main Project/ML algorithms'

In [1]:
import matplotlib.pyplot as plt
import torch
import torchvision

from torch import nn
from torchvision import transforms
# import helper_functions as set_seeds

In [21]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


In [ ]:
# # Identify and delete corrupted image in each of the folder file
# for folder_path in folder_paths:
#   for filename in os.listdir(folder_path):
#       try:
#           image = Image.open(os.path.join(folder_path, filename))
#       except PIL.UnidentifiedImageError as e:
#           print(f"Error in file {filename}: {e}")
#           os.remove(os.path.join(folder_path, filename))
#           print(f"Removed file {filename}")

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [4]:
from google.colab import files
src = list(files.upload().values())[0]
open('helper_functions.py','wb').write(src)
# from helper_functions import set_seeds

Saving helper_functions.py to helper_functions.py


6412

In [6]:
# 1. Get pretrained weights for ViT-Base
pretrained_vit_weights_req = torchvision.models.ViT_B_16_Weights.DEFAULT

# 2. Setup a ViT model instance with pretrained weights
pretrained_vit_req = torchvision.models.vit_b_16(weights=pretrained_vit_weights_req).to(device)

# 3. Freeze the base parameters
for val_ in pretrained_vit_req.parameters():
    val_.requires_grad = False

# 4. Change the classifier head
required_class_names = ['1','0']

# set_seeds()
pretrained_vit_req.heads = nn.Linear(in_features=768, out_features=len(required_class_names)).to(device)
# pretrained_vit_req # uncomment for model output

In [7]:
pip install torchinfo

In [8]:
from torchinfo import summary

# Print a summary using torchinfo (uncomment for actual output)
summary(model=pretrained_vit_req,
        input_size=(32, 3, 224, 224), # (batch_size, color_channels, height, width)
        # col_names=["input_size"], # uncomment for smaller output
        # col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]
)

Layer (type (var_name))                                      Output Shape         Param #
VisionTransformer (VisionTransformer)                        [32, 2]              768
├─Conv2d (conv_proj)                                         [32, 768, 14, 14]    (590,592)
├─Encoder (encoder)                                          [32, 197, 768]       151,296
│    └─Dropout (dropout)                                     [32, 197, 768]       --
│    └─Sequential (layers)                                   [32, 197, 768]       --
│    │    └─EncoderBlock (encoder_layer_0)                   [32, 197, 768]       (7,087,872)
│    │    └─EncoderBlock (encoder_layer_1)                   [32, 197, 768]       (7,087,872)
│    │    └─EncoderBlock (encoder_layer_2)                   [32, 197, 768]       (7,087,872)
│    │    └─EncoderBlock (encoder_layer_3)                   [32, 197, 768]       (7,087,872)
│    │    └─EncoderBlock (encoder_layer_4)                   [32, 197, 768]       (7,087,872)
│ 

In [ ]:
pwd

'/Users/kiranbijoy/Sem_3/Main Project/ML algorithms'

In [22]:
# Setup directory paths to train and test images
# train_dir = '/content/drive/MyDrive/Sen1Flood11_New_Dataset/train/'

train_folder = '/content/drive/MyDrive/Sen1Flood11_New_Dataset/train/'
test_folder = '/content/drive/MyDrive/Sen1Flood11_New_Dataset/test/'
valid_folder = '/content/drive/MyDrive/Sen1Flood11_New_Dataset/valid/'


# train_dir = '/Users/kiranbijoy/Sem_3/Main Project/ML algorithms/train/'
# test_dir = '/Users/kiranbijoy/Sem_3/Main Project/ML algorithms/test/'
# valid_dir = '/Users/kiranbijoy/Sem_3/Main Project/ML algorithms/valid/'

In [23]:
# Get automatic transforms from pretrained ViT weights
pretrained_vit_transforms_val = pretrained_vit_weights_req.transforms()
print(pretrained_vit_transforms_val)

ImageClassification(
    crop_size=[224]
    resize_size=[256]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BILINEAR
)


## And now the data is transformed, converting the images into DataLoaders using the create_dataloaders()

In [24]:
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

NUM_WORKERS = os.cpu_count()

def create_dataloaders_function( train_dir: str, test_dir: str, transform: transforms.Compose, batch_size: int, num_workers: int=NUM_WORKERS ):

  # Use ImageFolder to create train and test datasets
  train_data = datasets.ImageFolder(train_dir, transform=transform)
  test_data = datasets.ImageFolder(test_dir, transform=transform)

  # Get class names
  class_names = train_data.classes
  print('***************',class_names)


  # converting the images into data loaders
  train_dataloader = DataLoader( train_data, batch_size=batch_size, shuffle=True, num_workers=num_workers, pin_memory=True, )
  test_dataloader = DataLoader( test_data, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=True, )

  return train_dataloader, test_dataloader, class_names

In [25]:
# Setup dataloaders
train_dataloader_pretrained, test_dataloader_pretrained, class_names = create_dataloaders_function(train_dir=train_folder,
                                                                                                  test_dir=test_folder, transform=pretrained_vit_transforms_val,
                                                                                                     batch_size=32)

*************** ['0', '1']


In [30]:
from google.colab import files
src = list(files.upload().values())[0]
open('engine.py','wb').write(src)
import engine

Saving engine.py to engine (1).py


In [27]:
# "per_gpu_batch_size": (16, 64),
#   "weight_decay": (0, 0.3),
#   "learning_rate": (1e-5, 5e-5),
#   "warmup_steps": (0, 500),
#   "num_epochs": (2, 5)

In [28]:
# optimizer and loss function
required_optimizer1 = torch.optim.Adam(params=pretrained_vit_req.parameters(),
                             lr=3e-3,betas=(0.9,0.999),weight_decay=0.3)
required_loss_fn = torch.nn.CrossEntropyLoss()


In [ ]:
pretrained_vit_results = engine.train(model=pretrained_vit_req,
                                      train_dataloader=train_dataloader_pretrained,
                                      test_dataloader=test_dataloader_pretrained,
                                      optimizer=required_optimizer1, loss_fn=required_loss_fn,
                                      epochs=30, device=device)

  0%|          | 0/30 [00:00<?, ?it/s]

In [ ]:
required_optimizer2 = torch.optim.Adam(params=pretrained_vit_req.parameters(),
                             lr=2e-5)

In [ ]:
pretrained_vit_results = engine.train(model=pretrained_vit_req,
                                      train_dataloader=train_dataloader_pretrained,
                                      test_dataloader=test_dataloader_pretrained,
                                      optimizer=required_optimizer2,
                                      loss_fn=required_loss_fn,
                                      epochs=30,
                                      device=device)

In [ ]:
optimizer3 = torch.optim.Adam(params=pretrained_vit_req.parameters(),
                             lr=3e-5)


In [ ]:
pretrained_vit_results = engine.train(model=pretrained_vit_req,
                                      train_dataloader=train_dataloader_pretrained,
                                      test_dataloader=test_dataloader_pretrained,
                                      optimizer=optimizer3,
                                      loss_fn=required_loss_fn,
                                      epochs=50,
                                      device=device)

In [ ]:
optimizer4 = torch.optim.Adam(params=pretrained_vit_req.parameters(),
                             lr=5e-5)


In [ ]:
pretrained_vit_results = engine.train(model=pretrained_vit_req,
                                      train_dataloader=train_dataloader_pretrained,
                                      test_dataloader=test_dataloader_pretrained,
                                      optimizer=optimizer4,
                                      loss_fn=required_loss_fn,
                                      epochs=50,
                                      device=device)